# Curve Fitting
### Predicting the completion date of an engine recall based on historical engine replacement rates. 

The goal with this project was to determine a method for predicting the date a recall will be completed based on the information available. The information that was provided was: 
- The recall start date
- An estimated number of engines that were originally impacted by the recall
- Orders for replacement engines since the start date
- An estimated maximum monthly capacity for delivering replacements (whereever the bottleneck was in the supply chain)

A prediction interval was used using curve fitting to generate a 'best case scenario' and derivatives to generate a 'worst case scenario'

### Bottom Line Results

The final output of this project is a date interval within which the recall is expected to be completed. 

In [72]:
Image(url='https://github.com/evanlynch/portfolio/blob/master/imgs/RCP_Chart.png?raw=true')

In [67]:
print('Best Case End Date:',PredMin)
print('Worst Case End Date:',PredMax)

Best Case End Date: May 26,2019
Worst Case End Date: Aug 10,2019


In [70]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
import datetime
import matplotlib.pyplot as plt
import chartify as cfy
from IPython.display import Image

##### Inputs

In [79]:
#starting population of engines that need to be replaced at t=0
startDefectBalance = 150000

#the recall start date
startDate = "4/1/2018"

#estimated monthly max capacity for delivering replacements is 12k
monthlyCapacity = -12000
avgDailyCapacity = monthlyCapacity/30.42

#orders for replacement engines since the start date
df = pd.read_csv('Recall_Completion_Prediction/ReplacementOrders.csv',usecols=['Fill Date','Qty'])
df.head(10)

,Fill Date,Qty
0,5/31/2018,849
1,5/31/2018,265
2,6/6/2018,337
3,6/11/2018,166
4,6/13/2018,15
5,6/13/2018,88
6,6/13/2018,49
7,6/13/2018,25
8,6/14/2018,500
9,6/15/2018,500


##### Data Pre-Processing

In [80]:
#convert dates to a date delta and Qty to a remaining balance
df.Qty = df.Qty*-1
dfStart = pd.DataFrame({'Fill Date':startDate,'Qty':startDefectBalance,'ACOM':''},index=[999])
df = pd.concat([df,dfStart],sort=True)
startDate = datetime.datetime(2018, 4, 1)
df['Fill Date'] =  pd.to_datetime(df['Fill Date'])
df =df.set_index(['Fill Date'])

def removeDays(x):
    x['dateDelta'] = x['dateDelta'].days
    return x

#create new date starting from t=0
df['tvalue'] = df.index
df['dateDelta'] = df['tvalue']-startDate
df = df.apply(lambda x: removeDays(x),axis=1)
df = df.sort_values(['tvalue'],ascending=True)

#roll up orders to the daily level
df = df.groupby('dateDelta').sum()

#get current balance of defective engines
df['balance'] = df.Qty.cumsum()
df = df[['balance']]
df = df.reset_index()
print('Defect Balance Over Time')
df.head(n=10)

Defect Balance Over Time


,dateDelta,balance
0,0,150000
1,60,148886
2,66,148549
3,71,148383
4,73,148206
5,74,147706
6,75,147206
7,80,146706
8,81,145674
9,82,145567


In [81]:
ch = cfy.Chart(blank_labels=True)#, x_axis_type='datetime')
ch.set_title("Defect Balance Over Time")
ch.set_subtitle("As orders are placed, the defect balance will decrease.")
ch.plot.line(
    data_frame=df,
    x_column='dateDelta',
    y_column='balance')
ch.axes.set_yaxis_range(0,155000)
ch.axes.set_xaxis_range(0,800)
ch.show('html')

#### The plot above follows a curve similar to the function below. Estimate the parameters of that function

$f(x) = b_{0} - \frac{1}{b_{1}}x^2 \ \ \ \ \ \ \ \ \{x>0\}$

In [13]:
#the function f above
def func(x, b0, b1):
     return b0-(1/b1)*(x**2)

In [14]:
#f in terms of x
def getX(y, b0, b1):
    return np.sqrt((y-b0)*-b1)

In [15]:
#derivative of f
def getDerivative(x, b0, b1):
    return -(2/b1)*x

In [17]:
#fiting the curve
xdata=df.dateDelta
ydata=df.balance
popt, pcov = curve_fit(func, xdata, ydata)
xIntercept = getX(0,*popt)

In [82]:
#parameter estimates
print('b0:',popt[0])
print('b1:',popt[1])

b0: 150617.2977231807
b1: 1.1731030147676897


In [37]:
#using estimates to plot f
MoreDays = pd.Series(np.linspace(300,xIntercept,300),index=np.linspace(300,xIntercept,300))
xPlusMoreDays = xdata.append(MoreDays)
CF_results = pd.concat([xPlusMoreDays,func(xPlusMoreDays, *popt)],axis=1).rename({0:'x',1:'y'},axis=1)

In [38]:
#if modeled replacement rate exceeds the defined capacity rate (point at which f' exceeds capacity), use f' to find latest date recall could be completed
for i in range(0,int(xIntercept)):
    instReplacementRate = getDerivative(i,*popt)
    if instReplacementRate < avgDailyCapacity:
        y_CC = func(i,*popt)
        x_CC = getX(y_CC,*popt)
        slope = instReplacementRate
        yIntercept = y_CC-(slope*x_CC)
        break

#define the line
def capacityConstraint(x,intercept,slope):
    return intercept+(slope*x)
def getXCC(y,intercept,slope):
    return (y-intercept)/slope

xInterceptCC = getXCC(0,yIntercept,slope)

#create the capacity constraint
cap_x = pd.Series(np.linspace(x_CC,xInterceptCC,xInterceptCC-x_CC))
cap_y = pd.Series(capacityConstraint(x,yIntercept,slope))
der_results = pd.concat([cap_x,cap_y],axis=1).rename({0:'x',1:'y'},axis=1)

C:\Users\evan.lynch\AppData\Local\conda\conda\envs\my_root\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.


In [68]:
#plot defect balance, curve, and capacity constraint
cfy.color_palettes.create_palette(colors=['#d62728','#2ca02c','#1f77b4'],palette_type='categorical',name='custom palette')
ch = cfy.Chart(blank_labels=True)#, x_axis_type='datetime')
ch.set_title("Predicted Defect Balance Over Time")
ch.set_subtitle("As orders are placed, the defect balance will decrease.")
ch.style.set_color_palette('categorical', 'custom palette')
ch.plot.line(
    data_frame=der_results,
    x_column='x',
    y_column='y',
    line_dash='dashed')
ch.plot.line(
    data_frame=CF_results,
    x_column='x',
    y_column='y',
    line_dash='dashed')
ch.plot.line(
    data_frame=df,
    x_column='dateDelta',
    y_column='balance')
ch.axes.set_yaxis_range(0,150000)
ch.axes.set_xaxis_range(0,800)
ch.show('html')

In [74]:
#calculate the cv_rmse for the fitted curve against the actual data
def cv_rmse(predicted,actual):
    error = (predicted-actual)**2
    rmse = np.sqrt(error.sum()/error.count())
    per_rmse_pred_ma = rmse/(actual.sum()/actual.count())
    return per_rmse_pred_ma

print('Curve Fit CV_RMSE:',cv_rmse(func(xdata,*popt),ydata))

Curve Fit CV_RMSE: 0.015001391706549377


In [63]:
#get Min estimated end date 
numDaysToEnd = min([getX(0,*popt),xInterceptCC])
PredMin = str((startDate + datetime.timedelta(numDaysToEnd)).date().strftime('%b %d,%Y'))
print('Min Estimated End Date: ',PredMin)

#get Max estimated end date 
numDaysToEnd = max([getX(0,*popt),xInterceptCC])
PredMax = str((startDate + datetime.timedelta(numDaysToEnd)).date().strftime('%b %d,%Y'))
print('Max Estimated End Date: ',PredMax)

Min Estimated End Date:  May 26,2019
Max Estimated End Date:  Aug 10,2019
